<a href="https://colab.research.google.com/github/oObelix/DataScience/blob/master/Jun_ML_Linear_Regression_HW_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Урок 11. Домашняя работа

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома, где будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from numpy.linalg import inv
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
!ls sample_data

3.10_non_linear.csv	     california_housing_train.csv  README.md
anscombe.json		     mnist_test.csv
california_housing_test.csv  mnist_train_small.csv


In [3]:
data = pd.read_csv('sample_data/3.10_non_linear.csv')
data.head()

,x_train,y_train
0,0.138368,0.838812
1,0.157237,0.889313
2,0.188684,1.430040
3,0.685553,1.717309
4,0.874237,2.032588


In [4]:
def generate_degrees(source_data: list, degree: int):
    """Функция, которая принимает на вход одномерный массив, а возвращает n-мерный
    
    Для каждой степени от 1 до  degree возводим x в эту степень
    """
    return np.array([
          source_data**n for n in range(1, degree + 1)  
    ]).T

def train_polynomial(degree, data):
    """Генерим данные, тренируем модель
    """
    
    X = generate_degrees(data['x_train'], degree)

    model = LinearRegression().fit(X, data['y_train'])
    y_pred = model.predict(X)

    error = r2_score(data['y_train'], y_pred)

    return error


In [5]:
def grid_search(max_degree, data):
  good_error = -np.inf
  good_degree = 1
  for degree in range(1, max_degree + 1):
    error = train_polynomial(degree, data)
    if error > good_error:
      good_error = error
      good_degree = degree
  return good_degree, good_error

degree, error = grid_search(20, data)
print('Cтепень полинома, где лучший r2-score = {degree}, \
      \nпри этом скор = {error}'.format(degree=degree, error=error))

Cтепень полинома, где лучший r2-score = 14,       
при этом скор = 0.916397086647856


**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисляемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [ ]:
class CustomLinearReg:
    def __init__(self):
        pass
    
    def fit(self, X, y):
      Ones = np.ones((X.shape[0],2))
      Ones[:,1:] = np.array(X).reshape(-1,1)
      X = Ones
      y = np.array(y).reshape(-1,1)
      w = inv((X.T).dot(X)).dot(X.T).dot(y)
      self.w1 = w[0][0]
      self.w2 = w[1][0]
    
    def predict(self, X):
      return self.w1 + self.w2*np.array(X)

In [ ]:
clr = CustomLinearReg()
clr.fit(data['x_train'], data['y_train'])
clr.predict(data['x_train'])

array([ 1.92263843,  1.91631157,  1.9057668 ,  1.73915947,  1.67589087,
        1.57255215,  1.54935366,  1.5430268 ,  1.49873878,  1.49030296,
        1.47554029,  1.45445075,  1.44812389,  1.35111203,  1.29206133,
        1.20137633,  1.08327494,  1.06851226,  1.0305511 ,  1.02211529,
        0.99258994,  0.98837203,  0.87237959,  0.85761691,  0.78591249,
        0.76271401,  0.70788121,  0.70366331,  0.65515738,  0.59188877,
        0.59188877,  0.58556191,  0.56447238,  0.51596645,  0.44637098,
        0.43582621,  0.41895459,  0.41262772,  0.31350691,  0.25023831,
        0.23969354,  0.20595028,  0.18907865,  0.15322644,  0.14268168,
        0.12159214,  0.08784889, -0.0260346 , -0.03236146, -0.03236146])

In [ ]:
# Для проверки подсчитаем тот же массив ссредствами sklearn LinearRegression
# и убедимся, что массивы похожи или совпадают.
model = LinearRegression().fit(np.array(data['x_train']).reshape(-1, 1),
                               data['y_train'])
model.predict(np.array(data['x_train']).reshape(-1, 1))

array([ 1.92263843,  1.91631157,  1.9057668 ,  1.73915947,  1.67589087,
        1.57255215,  1.54935366,  1.5430268 ,  1.49873878,  1.49030296,
        1.47554029,  1.45445075,  1.44812389,  1.35111203,  1.29206133,
        1.20137633,  1.08327494,  1.06851226,  1.0305511 ,  1.02211529,
        0.99258994,  0.98837203,  0.87237959,  0.85761691,  0.78591249,
        0.76271401,  0.70788121,  0.70366331,  0.65515738,  0.59188877,
        0.59188877,  0.58556191,  0.56447238,  0.51596645,  0.44637098,
        0.43582621,  0.41895459,  0.41262772,  0.31350691,  0.25023831,
        0.23969354,  0.20595028,  0.18907865,  0.15322644,  0.14268168,
        0.12159214,  0.08784889, -0.0260346 , -0.03236146, -0.03236146])

**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid) с помощью функции `train_test_split` из библиотеки `sklearn`
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [6]:
boston_dataset = load_boston()

X = boston_dataset.data
y = boston_dataset.target

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, 
                                                      random_state=42)

reg = LinearRegression().fit(X_train, y_train)
y_X_valid_pred = reg.predict(X_valid)

r2_score(y_valid, y_X_valid_pred)

0.6687594935356307

In [13]:
# z-преобразования на уже разбитом множестве
scaler_X = StandardScaler()
X_train_z = scaler_X.fit_transform(X_train)
X_valid_z = scaler_X.transform(X_valid)

scaler_y = StandardScaler()
y_train_z = scaler_y.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
y_valid_z = scaler_y.transform(y_valid.reshape(-1, 1)).reshape(-1)

In [10]:
reg = LinearRegression().fit(X_train_z, y_train_z)
y_pred_X_valid_z = reg.predict(X_valid_z)

r2_score(y_valid_z, y_pred_X_valid_z)

0.668759493535632

### Вывод: Z-преобразование не влияет на качество модели по r2-score.